### causal-learn

In [1]:
import numpy as np

def gen_coef():
    return np.random.uniform(1, 3)

np.random.seed(0)
sample_size, loc, scale = 1000, 0.0, 1.0
L1 = np.random.normal(loc=loc, scale=scale, size=sample_size)
L2 = np.random.normal(loc=loc, scale=scale, size=sample_size)
L3 = np.random.normal(loc=loc, scale=scale, size=sample_size)
X1 = gen_coef() * L1 + gen_coef() * L2 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X2 = gen_coef() * X1 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X3 = gen_coef() * X2 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X4 = gen_coef() * X1 + gen_coef() * L3 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X5 = gen_coef() * X3 + gen_coef() * L3 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X6 = gen_coef() * L1 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X7 = gen_coef() * L2 + gen_coef() * L3 + gen_coef() * X6 + np.random.normal(loc=loc, scale=scale,
                                                                            size=sample_size)
data = np.array([L1,L2,L3,X1, X2, X3, X4, X5, X6, X7]).T
data

array([[  1.76405235,   0.55596268,  -1.53292105, ...,  37.12065837,
          2.19728166,   1.71483045],
       [  0.40015721,   0.89247389,  -1.71197016, ...,  25.11396725,
          2.26405252,   0.69541575],
       [  0.97873798,  -0.42231482,   0.04613506, ...,  -1.2606711 ,
          0.40624366,  -0.37687517],
       ...,
       [  0.0941923 ,   0.15843385,  -0.19240421, ...,   1.57758945,
         -0.32128275,  -1.02344089],
       [ -1.14761094,  -1.14190142,  -1.21251574, ..., -29.52258499,
         -3.3421349 ,  -8.57431905],
       [ -0.35811408,  -1.31097037,  -0.08059852, ..., -19.45826394,
          1.03476209,   0.62413589]])

### chaoxi

In [2]:
import numpy as np
import pandas as pd

N = 100000
loc, scale = 0, 1

IQ = np.random.normal(loc=loc, scale=scale, size=sample_size)
edu = 0.5 * IQ + np.random.normal(loc=loc, scale=scale+1, size=sample_size)
earnings = 0.3 * IQ + 0.4 * edu + np.random.normal(loc=loc, scale=scale+2, size=sample_size)
art = 1.2 * edu + 0.6 * earnings + np.random.normal(loc=loc, scale=scale+3, size=sample_size)

data = pd.DataFrame({'IQ':IQ, 'edu':edu, 'earnings':earnings, 'art':art})

In [3]:
import statsmodels.formula.api as smf

model = smf.ols(formula='earnings ~ IQ + edu', data=data)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     48.83
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           5.87e-21
Time:                        17:14:07   Log-Likelihood:                -2538.2
No. Observations:                1000   AIC:                             5082.
Df Residuals:                     997   BIC:                             5097.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0946      0.097     -0.973      0.3

In [4]:
import statsmodels.formula.api as smf

model = smf.gls(formula='earnings ~ IQ + edu', data=data)
result = model.fit()
print(result.summary())

                            GLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.089
Model:                            GLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     48.83
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           5.87e-21
Time:                        17:14:07   Log-Likelihood:                -2538.2
No. Observations:                1000   AIC:                             5082.
Df Residuals:                     997   BIC:                             5097.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0946      0.097     -0.973      0.3

In [5]:
import statsmodels.formula.api as smf

model = smf.ols(formula='earnings ~ IQ + edu + art', data=data)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     120.1
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           2.14e-66
Time:                        17:14:07   Log-Likelihood:                -2430.5
No. Observations:                1000   AIC:                             4869.
Df Residuals:                     996   BIC:                             4889.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0861      0.087     -0.985      0.3

In [6]:
import statsmodels.formula.api as smf

model = smf.gls(formula='earnings ~ IQ + edu + art', data=data)
result = model.fit()
print(result.summary())

                            GLS Regression Results                            
Dep. Variable:               earnings   R-squared:                       0.266
Model:                            GLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     120.1
Date:                Sun, 04 Sep 2022   Prob (F-statistic):           2.14e-66
Time:                        17:14:07   Log-Likelihood:                -2430.5
No. Observations:                1000   AIC:                             4869.
Df Residuals:                     996   BIC:                             4889.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0861      0.087     -0.985      0.3

gls 和 ols 的系数拟合是一样的，这可能是因为没有异方差的问题？但是现在每个特征的方差都不一样，结果还是之前的结论

当R和python的结果不一样时，应该选择哪个更合适？

### xingming

In [7]:
import numpy as np
import pandas as pd

N = 100000
loc, scale = 0, 1

C1 = np.random.normal(loc=loc, scale=scale, size=sample_size)
C2 = C1 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C3 = C2 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C4 = C3 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C5 = C4 + np.random.normal(loc=loc, scale=scale, size=sample_size)
C6 = C5 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X1 = np.random.normal(loc=loc, scale=scale, size=sample_size)
X2 = np.random.normal(loc=loc, scale=scale, size=sample_size)
B = C1 + C2 + C3 + C4 + C5 + C6
A = B + C1 + C2 + C3 + C4 + C5 + C6 + X1 + X2

X3 = np.random.normal(loc=loc, scale=scale, size=sample_size)
X4 = np.random.normal(loc=loc, scale=scale, size=sample_size) + B 
E = B + X3 + X4

X5 = C6 + np.random.normal(loc=loc, scale=scale, size=sample_size)
X6 = np.random.normal(loc=loc, scale=scale, size=sample_size)
F = B + X5 + X6

data = pd.DataFrame({'C1':C1, 'C2':C2, 'C3':C3, 'C4':C4, 'C5':C5, 'C6':C6,
                     'X1':X1, 'X2':X2, 'X3':X3, 'X4':X4, 'X5':X5, 'X6':X6,
                     'A':A, 'B':B, 'E':E, 'F':F})

#### 对B进行回归

In [8]:
import statsmodels.formula.api as smf

model = smf.ols(formula='B ~ C1 + C2 + C3 + C4 + C5 + C6', data=data) ### 直接拟合数据生成方程 （准！) 无多重共线性
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      B   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.784e+32
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:07   Log-Likelihood:                 31677.
No. Observations:                1000   AIC:                        -6.334e+04
Df Residuals:                     993   BIC:                        -6.331e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.832e-15   1.35e-16    -13.548      0.0

#### 对A进行回归

In [9]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B + C1 + C2 + C3 + C4 + C5 + C6', data=data) ### 仅考虑后门变量 但存在多重共线性
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 2.782e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                -1765.1
No. Observations:                1000   AIC:                             3544.
Df Residuals:                     993   BIC:                             3579.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0571      0.045      1.263      0.2

In [10]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B', data=data) ### 解决多重共线性  （准！）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.663e+05
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                -1769.7
No. Observations:                1000   AIC:                             3543.
Df Residuals:                     998   BIC:                             3553.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0481      0.045      1.067      0.2

In [11]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B + C1 + C2 + C3 + C4 + C5 + C6 + X1 + X2', data=data) ### 直接拟合数据生成方程 （不准）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.022e+33
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                 31263.
No. Observations:                1000   AIC:                        -6.251e+04
Df Residuals:                     991   BIC:                        -6.246e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -8.327e-16   2.05e-16     -4.062      0.0

In [12]:
import statsmodels.formula.api as smf

model = smf.ols(formula='A ~ B + C1 + C2 + C3 + C4 + C5', data=data)  ### 当后门变量未充分考虑时
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      A   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 2.782e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                -1765.1
No. Observations:                1000   AIC:                             3544.
Df Residuals:                     993   BIC:                             3579.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0571      0.045      1.263      0.2

#### 对E进行回归

In [13]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='E ~ B', data=data)  ### 仅考虑后门变量 但后门变量为空时 （准！）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      E   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.672e+05
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                -1765.4
No. Observations:                1000   AIC:                             3535.
Df Residuals:                     998   BIC:                             3545.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0154      0.045     -0.344      0.7

In [14]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='E ~ B + X3 + X4', data=data)  ### 直接拟合数据生成方程 （准！) 无多重共线性
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      E   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.226e+33
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                 31750.
No. Observations:                1000   AIC:                        -6.349e+04
Df Residuals:                     996   BIC:                        -6.347e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -5.551e-17   1.25e-16     -0.444      0.6

#### 对F进行回归

In [15]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='F ~ B + X6', data=data)  ### 仅考虑后门变量 且后门变量为1个时 （准？）照理说应该不准，B和X6存在共线性
result = model.fit()  ### 可能是因为共线性较弱？
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      F   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                 3.244e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                -1762.3
No. Observations:                1000   AIC:                             3531.
Df Residuals:                     997   BIC:                             3545.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0141      0.045     -0.316      0.7

In [16]:
import statsmodels.formula.api as smf  

model = smf.ols(formula='F ~ B + X6 + X7', data=data)  ### 直接拟合数据生成方程 （不准）
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      F   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                 2.165e+04
Date:                Sun, 04 Sep 2022   Prob (F-statistic):               0.00
Time:                        17:14:08   Log-Likelihood:                -1761.3
No. Observations:                1000   AIC:                             3531.
Df Residuals:                     996   BIC:                             3550.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0185      0.045     -0.412      0.6